In [2]:
import pandas as pd
df = pd.read_csv("parfums.csv")

In [6]:
# Filter DataFrames by gender
df_homme = df[df["gender"].str.capitalize() == "Homme"]
df_femme = df[df["gender"].str.capitalize() == "Femme"]
df_mixte = df[df["gender"].str.capitalize() == "Mixte"]

# Print summary of filtered data
print(f"Nombre de parfums pour Homme : {len(df_homme)}")
print(f"Nombre de parfums pour Femme : {len(df_femme)}")
print(f"Nombre de parfums pour Mixte : {len(df_mixte)}")

# Function to get frequencies from specified columns (for scents or facettes)
def get_frequencies(df, columns):
    if df.empty:
        return pd.Series(dtype=int)
    return (df[columns]
            .stack()
            .str.split(';', expand=True)
            .stack()
            .str.strip()
            .value_counts())

# Data analysis for the "Analysis" sheet
# 1. Brand frequencies (overall and by gender)
brand_freq = df['brand'].value_counts().reset_index(name='count').rename(columns={'index': 'brand'})
brand_freq_homme = df_homme['brand'].value_counts().reset_index(name='count').rename(columns={'index': 'brand_homme'})
brand_freq_femme = df_femme['brand'].value_counts().reset_index(name='count').rename(columns={'index': 'brand_femme'})
brand_freq_mixte = df_mixte['brand'].value_counts().reset_index(name='count').rename(columns={'index': 'brand_mixte'})

# 2. Facettes frequencies (overall and by gender)
facettes_freq = get_frequencies(df, ['facettes']).reset_index(name='count').rename(columns={'index': 'facette'})
facettes_freq_homme = get_frequencies(df_homme, ['facettes']).reset_index(name='count').rename(columns={'index': 'facette_homme'})
facettes_freq_femme = get_frequencies(df_femme, ['facettes']).reset_index(name='count').rename(columns={'index': 'facette_femme'})
facettes_freq_mixte = get_frequencies(df_mixte, ['facettes']).reset_index(name='count').rename(columns={'index': 'facette_mixte'})

# 3. Family frequencies
family_freq = df['family'].value_counts().reset_index(name='count').rename(columns={'index': 'family'})

# 4. Concentration frequencies
concentration_freq = df['concentration'].value_counts().reset_index(name='count').rename(columns={'index': 'concentration'})

# 5. Scent frequencies (overall and by gender)
scent_freq_all = get_frequencies(df, ['notes_tete', 'notes_coeur', 'notes_fond']).reset_index(name='count').rename(columns={'index': 'scent'})
scent_freq_homme = get_frequencies(df_homme, ['notes_tete', 'notes_coeur', 'notes_fond']).reset_index(name='count').rename(columns={'index': 'scent_homme'})
scent_freq_femme = get_frequencies(df_femme, ['notes_tete', 'notes_coeur', 'notes_fond']).reset_index(name='count').rename(columns={'index': 'scent_femme'})
scent_freq_mixte = get_frequencies(df_mixte, ['notes_tete', 'notes_coeur', 'notes_fond']).reset_index(name='count').rename(columns={'index': 'scent_mixte'})

# 6. Scent frequencies by note
scent_freq_tete = get_frequencies(df, ['notes_tete']).reset_index(name='count').rename(columns={'index': 'scent_tete'})
scent_freq_coeur = get_frequencies(df, ['notes_coeur']).reset_index(name='count').rename(columns={'index': 'scent_coeur'})
scent_freq_fond = get_frequencies(df, ['notes_fond']).reset_index(name='count').rename(columns={'index': 'scent_fond'})

# Create a list of analysis DataFrames with titles and colors
analysis_dfs = [
    {'title': 'Marques', 'data': brand_freq, 'header_color': '#4B8BBE'},  # Blue
    {'title': 'Marques (Homme)', 'data': brand_freq_homme, 'header_color': '#5DADE2'},  # Light Blue
    {'title': 'Marques (Femme)', 'data': brand_freq_femme, 'header_color': '#F4D03F'},  # Yellow
    {'title': 'Marques (Mixte)', 'data': brand_freq_mixte, 'header_color': '#52BE80'},  # Green
    {'title': 'Familles', 'data': family_freq, 'header_color': '#3498DB'},  # Sky Blue
    {'title': 'Facettes', 'data': facettes_freq, 'header_color': '#8E44AD'},  # Purple
    {'title': 'Facettes (Homme)', 'data': facettes_freq_homme, 'header_color': '#9B59B6'},  # Light Purple
    {'title': 'Facettes (Femme)', 'data': facettes_freq_femme, 'header_color': '#D7BDE2'},  # Very Light Purple
    {'title': 'Facettes (Mixte)', 'data': facettes_freq_mixte, 'header_color': '#BB8FCE'},  # Medium Purple
    {'title': 'Concentrations', 'data': concentration_freq, 'header_color': '#E74C3C'},  # Red
    {'title': 'Senteurs', 'data': scent_freq_all, 'header_color': '#68A225'},  # Olive Green
    {'title': 'Senteurs (Homme)', 'data': scent_freq_homme, 'header_color': '#FFD166'},  # Light Yellow
    {'title': 'Senteurs (Femme)', 'data': scent_freq_femme, 'header_color': '#F28C38'},  # Orange
    {'title': 'Senteurs (Mixte)', 'data': scent_freq_mixte, 'header_color': '#E67E22'},  # Carrot
    {'title': 'Senteurs (Tête)', 'data': scent_freq_tete, 'header_color': '#1ABC9C'},  # Turquoise
    {'title': 'Senteurs (Coeur)', 'data': scent_freq_coeur, 'header_color': '#2ECC71'},  # Emerald
    {'title': 'Senteurs (Fond)', 'data': scent_freq_fond, 'header_color': '#27AE60'}  # Dark Green
]

# Write to Excel with separate sheets
with pd.ExcelWriter("parfums_par_genre.xlsx", engine="xlsxwriter") as writer:
    # Write main DataFrame sheet first
    df.to_excel(writer, sheet_name="All Parfums", index=False)
    
    # Write data sheets
    df_homme.to_excel(writer, sheet_name="Homme", index=False)
    df_femme.to_excel(writer, sheet_name="Femme", index=False)
    df_mixte.to_excel(writer, sheet_name="Mixte", index=False)
    
    # Write analysis sheet
    workbook = writer.book
    worksheet = workbook.add_worksheet('Analysis')
    col_start = 0  # Starting column for each table
    
    for analysis in analysis_dfs:
        # Define header format with color
        header_format = workbook.add_format({
            'bold': True,
            'bg_color': analysis['header_color'],
            'border': 1,
            'align': 'center',
            'valign': 'vcenter',
            'text_wrap': True
        })
        
        # Write section title
        worksheet.write(0, col_start, analysis['title'], header_format)
        
        # Write data with headers
        for idx, column in enumerate(analysis['data'].columns):
            worksheet.write(1, col_start + idx, column, header_format)
        for row_idx, row_data in enumerate(analysis['data'].itertuples(index=False), start=2):
            for col_idx, value in enumerate(row_data):
                worksheet.write(row_idx, col_start + col_idx, value)
        
        # Adjust column width
        for idx, column in enumerate(analysis['data'].columns):
            max_length = max(
                analysis['data'][column].astype(str).map(len).max(),
                len(column)
            )
            worksheet.set_column(col_start + idx, col_start + idx, max_length + 2)
        
        # Move to the next table's starting column (add 1 column gap)
        col_start += len(analysis['data'].columns) + 1

    # Freeze the top row for easier scrolling
    worksheet.freeze_panes(2, 0)

print("Excel file with 'All Parfums' and updated analysis sheet created successfully.")

Nombre de parfums pour Homme : 582
Nombre de parfums pour Femme : 1192
Nombre de parfums pour Mixte : 277
Excel file with 'All Parfums' and updated analysis sheet created successfully.
